### 1. Dataset

Los datos de origen son proporcionados en un archivos csv:

* udfs: dataset con datos de operaciones financieras.


In [62]:
import pandas as pd
import numpy as np

udfs = pd.read_csv("udfs.csv",sep=";")
udfs

,nb,contract,udf_ref,fmly,grp,type,country,udf_name,num_value,string_value,date_value,data_timestamp_part,data_date_part,source_system
0,444444,3333,28786653,IRD,LN_BR,NaN,ESP,M_CCY,NaN,NaN,NaN,20201128041303,2020-12-30,Mx3EU
1,2222222,2222222,2222222,IRD,IRS,NaN,ESP,M_CRDTCHRG,30.0,NaN,NaN,20210203032054,2020-12-30,Mx3EU
2,2222222,2222222,2222222,IRD,IRS,NaN,ESP,M_SELLER,NaN,LB_TLECLER,NaN,20210203032054,2020-12-30,Mx3EU
3,2222222,2222222,2222222,IRD,IRS,NaN,ESP,M_LIQDTYCHRG,50.0,NaN,NaN,20210203032054,2020-12-30,Mx3EU
4,2222222,2222222,2222222,IRD,IRS,NaN,ESP,M_MVA,20.0,NaN,NaN,20210203032054,2020-12-30,Mx3EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,10000009,10000009,10000009,IRD,BOND,FWD,ESP,M_SUCURSAL,NaN,1999,NaN,20210203032054,2020-12-30,Mx3EU
382,10000009,10000009,10000009,IRD,BOND,FWD,ESP,M_RVA,0.0,NaN,NaN,20210203032054,2020-12-30,Mx3EU
383,10000009,10000009,10000009,IRD,BOND,FWD,ESP,M_DIRECTIAV,0.0,NaN,NaN,20210203032054,2020-12-30,Mx3EU
384,10000009,10000009,10000009,IRD,BOND,FWD,ESP,M_CLIENT,NaN,NaN,NaN,20210203032054,2020-12-30,Mx3EU


### 2. Columnas y significado:

* nb: número de referencia de la operación.
* contract: identificador de contrato.
* udf_ref: identificador de operación de trading.
* fmly: familia a la que pertenece la operación financiera.
* grp: grupo al que pertenece la operación financiera.
* type: tipo de operación financiera.
* country: país de origen de la operación.
* udf_name: campo informado en el registro.
* num_value: valor numérico.
* string_value: valor de cadena de caracteres.
* date_value: valor de fecha.
* data_timestamp_part: marca temporal.
* data_date_part: fecha en la que se almacena la información.
* source_system: fuente de los datos.


### 3. Descripción del problema:

Si hacemos una visión general a nuestro conjunto de datos, podemos observar como hay hasta 10 registros (filas) para cada valor de *nb*, donde cada registro solo da información para un valor de *udf_name*. Esto es un gasto innecesario de almacenamiento y computación, además de complicar los futuros cálculos derivados de estos datos. Por esta razón, necesitamos convertir estos registros con el mismo *nb* a un solo registro.

Nuestro dataframe final tendrá que contener las siguientes columnas: `nb, M_CCY, M_CLIENT, M_CRDTCHRG, M_DIRECTIAV, M_DISCMARGIN, M_LIQDTYCHRG, M_MVA, M_RVA, M_SELLER, M_SUCURSAL`

* nb: debe contener el número de referencia de la operación.
* M_CLIENT, M_SELLER, M_CCY, M_SUCURSAL: deben mapear el valor de *string_value*
* M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA: deben mapear el valor de *num_value*


Una vez tengamos este resultado, necesitaremos eliminar las operaciones que no tengan informados ninguno de los siguientes campos:

M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, M_MVA, M_RVA, M_SELLER

No informados en este caso significa que o son valores nulos, vacíos o 0, en el caso de los campos numéricos.

In [137]:
#Devuelvo un dataframe con las columnas que se piden
def dfFinalEmpty(udfs):
    #Copio el df
    udfsCopy = udfs[["nb","udf_name","string_value","num_value"]].copy()
    #Obtengo los valores unicos de la columna udf_name y lo paso a un tipo lista
    columnas = udfsCopy["udf_name"].unique().tolist()
    #Inserto en la lista el campo nb en la primera posicion
    columnas.insert(0,"nb")

    #Devuelvo un df vacio con las columnas
    return pd.DataFrame(columns=columnas)

#Devuelvo un dataframe con los valores no repetidos de la columna nb
def nbInsertVal(udfs):
    #Obtengo un df vacio
    dfEmpty = dfFinalEmpty(udfs)
    #Inserto en la columna nb los valores nb unicos de udfs
    dfEmpty["nb"] = udfs["nb"].unique()

    return dfEmpty

def darValor(udfs):
    #Obtengo un df con los nb
    df = nbInsertVal(udfs)

    #Lista con las columnas que son tipo string o tipo num
    str_val = ["M_CLIENT","M_SELLER", "M_CCY", "M_SUCURSAL"]
    num_val = ["M_DISCMARGIN", "M_DIRECTIAV", "M_LIQDTYCHRG", "M_CRDTCHRG", "M_MVA", "M_RVA"]

    #Una lista de las columnas que no se requieren
    delCol = list()

    #Bucle para recorrer cada fila del df (Lo hago con la columna nb para asi ahorrarme acceder a la posicion)
    for nb in df.nb:
        #Obtengo un df auxiliar que tenga solo los valores nb iguales.
        aux = udfs[udfs["nb"] == nb].loc[:,["nb", "udf_name", "num_value","string_value"]]
        #Bucle para recorrer las columnas del df
        for col in df.columns[1:]:
            value = "Error"

            #Si la columna tiene que ser tipo string
            if col in str_val:
                value = aux.loc[:,"string_value"][aux["udf_name"] == col].to_list()[0] if len(aux.loc[:,"string_value"][aux["udf_name"] == col].to_list()) > 0 else None

            #Si la columna tiene que ser tipo numerico
            if col in num_val:
                value = aux.loc[:,"num_value"][aux["udf_name"] == col].to_list()[0] if len(aux.loc[:,"num_value"][aux["udf_name"] == col].to_list()) > 0 else None

            #Si value mantiene su valor es que la columna no se nos pide. Almaceno la columna a eliminar
            if value == "Error" and col not in delCol: delCol.append(col)

            #Le asigno el valor al df
            df.loc[df[df.nb == nb].index,col] = value

    if len(delCol) > 0: df = df.drop(columns=delCol)
    return df

def eliminarNoInformados(udfs):
    #Columnas para comparar
    str_col = ["M_CLIENT","M_SELLER", "M_CCY", "M_SUCURSAL"]
    num_col = ["M_DISCMARGIN", "M_DIRECTIAV", "M_LIQDTYCHRG", "M_CRDTCHRG", "M_MVA", "M_RVA"]

    df = darValor(udfs)
    #df = df.astype({'M_DISCMARGIN':'float', "M_DIRECTIAV":'float', "M_LIQDTYCHRG":'float', "M_CRDTCHRG":'float', "M_MVA":'float', "M_RVA":'float'})
    df = df[ ~((df[str_col].isna().values.all(axis=1)) & (df[num_col] <= 0).all(axis=1))]
    #df = df[~(df[str_col].isna().values.all(axis=1)) & ~(df[num_col] <= 0).all(axis=1)]
    return df

eliminarNoInformados(udfs)

,nb,M_CCY,M_CRDTCHRG,M_SELLER,M_LIQDTYCHRG,M_MVA,M_SUCURSAL,M_RVA,M_DIRECTIAV,M_CLIENT,M_DISCMARGIN
0,444444,NaN,None,None,None,None,None,None,None,None,None
1,2222222,USD,30.0,LB_TLECLER,50.0,20.0,1999,0.0,0.0,CCMO,10.0
3,8216817,EUR,None,AMAM,None,None,None,None,0.0,None,None
4,14773283,NaN,10.0,NaN,10.0,10.0,5493,5.0,10.0,NaN,200.0
5,16719306,USD,None,AMAM,None,None,None,None,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...
57,20665177,NaN,20.0,LB_VSTAVRE,20.0,0.0,5493,0.0,1200.0,NaN,0.0
58,20665178,NaN,20.0,LB_VSTAVRE,20.0,0.0,5493,0.0,1200.0,NaN,100.0
59,556111214,USD,30.0,LB_TLECLER,50.0,20.0,1999,0.0,0.0,NaN,10.0
60,10000001,None,20.0,SELLER1,30.0,0.0,1999,0.0,0.0,NaN,10.0


### 4. Reto:

* Obtener un dataframe final que contenga las columnas indicadas, con un registro por *nb* y con los valores correctos mapeados.
* Las operaciones con los campos M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA, M_SELLER no informados no deben existir.
* Hacerlo de la manera más eficiente posible a nivel computacional.

**NOTA:** Cada uno de los pasos descritos en el problema pueden efectuarse en una sola línea.